In [12]:
from os import environ

import numpy as np
from pandas import DataFrame
from tools.interpretation import get_segments
from tools.trajectory import calculate_trajectory
from tools.enums import EMeasureUnits
from python_sdk.client import PyRogii

PROJECT_NAME = 'nsapegin (ft)'
WELL_NAME = 'Lateral1'
INTERPRETATION_NAME = 'Interpretation1'
measure_unit = EMeasureUnits.METER_FOOT


pr = PyRogii(
    client_id=environ.get('CLIENT_ID'),
    client_secret=environ.get('CLIENT_SECRET'),
    project_name=PROJECT_NAME,
    solo_username=environ.get('SOLO_USERNAME'),
    solo_password=environ.get('SOLO_PASSWORD'),
    papi_domain_name=environ.get('PAPI_DOMAIN_NAME')
)


def interpretation_dip_calculation():
    papi_well = pr.get_well(well_name=WELL_NAME)

    if not papi_well:
        print(f'Well "{papi_well["name"]}" not found')
        return

    well_trajectory = pr.get_well_trajectory(well_name=WELL_NAME)

    calculated_trajectory = calculate_trajectory(well_trajectory, papi_well, measure_unit=measure_unit)

    interpretation = pr.get_well_starred_interpretation(well_name=WELL_NAME)

    if not interpretation:
        interpretation = pr.get_well_interpretation(
            well_name=WELL_NAME, interpretation_name=INTERPRETATION_NAME
        )

    if not interpretation:
        print(f'Interpretation "{INTERPRETATION_NAME}" in the well "{WELL_NAME}" not found')
        return

    segments = get_segments(
        papi_well,
        assembled_segments=interpretation['segments'],
        assembled_horizons=interpretation['horizons'],
        calculated_trajectory=calculated_trajectory,
        measure_unit=measure_unit
    )

    _df = DataFrame(segments, columns=['md', 'dip'])
    np_array = _df.to_numpy()
    axe = np.arange(np_array[0, 0], np_array[-1, 0], 50.0)
    axe = np.unique(np.append(axe, np_array[:, 0]))
    axe.sort()
    interpolated_dip = np.interp(axe, np_array[:, 0], np_array[:, 1])
    _result_df = DataFrame((axe, interpolated_dip), index=['md', 'dip']).transpose()
    return _df, _result_df


df, result_df = interpretation_dip_calculation()
df

,md,dip
0,3439.650526,92.213120
1,3508.552800,98.565274
2,3537.508800,91.455664
3,3714.292800,91.674925
4,3979.684251,90.025220
5,4084.549663,90.623995
6,4221.449520,91.574348
7,4264.277649,91.092151
8,4313.438160,94.867358
9,4340.504400,92.041099


In [13]:
result_df

,0,1
0,3439.650526,92.213120
1,3489.650526,96.822659
2,3508.552800,98.565274
3,3537.508800,91.455664
4,3539.650526,91.458320
5,3589.650526,91.520334
6,3639.650526,91.582348
7,3689.650526,91.644361
8,3714.292800,91.674925
9,3739.650526,91.517298
